In [35]:
! pip install py_dotenv langchain langchain_groq langchain_community chromadb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 8.9 MB/s eta 0:00:0

In [24]:
import dotenv

dotenv.load_dotenv('/content/drive/MyDrive/Active Loop RAG/api.env')

True

In [25]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model_name='llama-3.1-70b-versatile'
)

In [26]:
response = llm.invoke('Thefirst person to land on the moon was..')

In [27]:
response.content

"Neil Armstrong. He stepped onto the moon's surface on July 20, 1969, during the Apollo 11 mission."

In [18]:
from langchain_community.document_loaders import WebBaseLoader
from textwrap import wrap

loader = WebBaseLoader('https://jobs.nike.com/job/R-32222?from=job%20search%20funnel')
page_data = loader.load().pop().page_content
print('\n'.join(wrap(page_data, width=220)))

Apply for Senior Machine Learning Engineer  Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative
SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing &
EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution
& LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability
InclusionIndigenous InclusionInternshipsData & AnalyticsSenior Machine Learning EngineerGdansk, PomeranianBecome a Part of the NIKE, I

In [28]:
from langchain_core.prompts import PromptTemplate

scraping_prompt = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of the company {company}.
    Your job is to extract the job postings and return them in JSON format
    contaning the following keys: `role,` `experience,` `skills` and `description.`
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

scraping_chain = scraping_prompt | llm
res = scraping_chain.invoke(
    {
        'page_data':page_data,
        'company':'Nike'
    }
)

In [29]:
print(res.content)

```
{
  "role": "Senior Machine Learning Engineer",
  "experience": "3+ years of experience in the field of ML Engineering or Software Engineering",
  "skills": [
    "Understanding of Machine Learning, its applications, and the lifecycle of an ML application in production",
    "Ability to significantly communicate, written, orally, and visually technical topics with peers",
    "Ability to write robust, maintainable, and extendable code in Python",
    "Experience working in and/or collaborating with a partial or fully distributed team",
    "Strong experiential understanding of data structures, algorithms, and data solutions",
    "Familiarity with frameworks such as Scikit-learn, PyTorch, Tensorflow, Spark, FastAPI or similar platforms and frameworks",
    "Experience with cloud architecture and technologies (preferably Amazon Web Services: ECR, SageMaker, Lambda, API Gateway)",
    "Familiarity with pipeline orchestration tools such as AirFlow",
    "Awareness of CI/CD pipelines a

In [31]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_response = json_parser.parse(res.content)
json_response

{'role': 'Senior Machine Learning Engineer',
 'experience': '3+ years of experience in the field of ML Engineering or Software Engineering',
 'skills': ['Understanding of Machine Learning, its applications, and the lifecycle of an ML application in production',
  'Ability to significantly communicate, written, orally, and visually technical topics with peers',
  'Ability to write robust, maintainable, and extendable code in Python',
  'Experience working in and/or collaborating with a partial or fully distributed team',
  'Strong experiential understanding of data structures, algorithms, and data solutions',
  'Familiarity with frameworks such as Scikit-learn, PyTorch, Tensorflow, Spark, FastAPI or similar platforms and frameworks',
  'Experience with cloud architecture and technologies (preferably Amazon Web Services: ECR, SageMaker, Lambda, API Gateway)',
  'Familiarity with pipeline orchestration tools such as AirFlow',
  'Awareness of CI/CD pipelines and containerization is preferr

In [32]:
type(json_response)

dict

In [34]:
import pandas as pd

portfolio_data = pd.read_csv('my_portfolio.csv')

In [36]:
import uuid
import chromadb

In [39]:
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
  for _, row in portfolio_data.iterrows():
    collection.add(
        documents=row['Techstack'],
        metadatas={
            'links': row['Links']
        },
        ids=[str(uuid.uuid4())]
    )

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 87.0MiB/s]


In [41]:
links = collection.query(
    query_texts=json_response['skills'],
    n_results=2
).get('metadatas', [])

In [42]:
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https

In [46]:
email_prompt = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are a business development executive at ABCQP Services. ABCQP Services is an AI & software consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools.
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
    process optimization, cost reduction, and heightened overall efficiency.
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase ABCQP Services's portfolio: {link_list}
    Remember you are a business development executive at ABCQP Services.
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)

In [47]:
email_chain = email_prompt | llm
res = email_chain.invoke(
    input={
        'job_description': json_response['description'],
        'link_list': links
    }
)

In [51]:
print(res.content)

Subject: Expert Machine Learning Solutions for Nike's AIML Team

Dear Hiring Manager,

I came across the job description for a Sr. Machine Learning Engineer within Nike's AIML team and was impressed by the company's commitment to developing sophisticated analytics systems. As a business development executive at ABCQP Services, I believe our team can provide the expertise and support needed to fulfill Nike's requirements.

Our company has a proven track record of delivering high-quality AI and software solutions that drive business growth and efficiency. We've empowered numerous enterprises with tailored solutions, resulting in scalability, process optimization, cost reduction, and heightened overall efficiency.

In particular, our expertise in machine learning and software engineering aligns with Nike's needs. We've developed and implemented various machine learning solutions using Python, as showcased in our portfolio: https://example.com/ml-python-portfolio. Additionally, our experie